---

# 📦 Model Dump & JSON

### 🎯 Intent

Serialize Pydantic v2 models to **dicts or JSON** for APIs, storage, or logging—while controlling fields and formats.

---

### 🧩 Core Functions

1. **🧰 `.model_dump()`**

   * Returns a Python dict.
   * Options: `include`, `exclude`, `exclude_none`, `exclude_unset`, `exclude_defaults`, `mode`.

2. **📤 `.model_dump_json()`**

   * Returns a JSON string.
   * Same filters as `.model_dump()`.

3. **🔄 Modes**

   * `mode="python"` → keeps rich types (`datetime`, `UUID`).
   * `mode="json"` → converts to JSON-friendly strings.

4. **🎯 Filtering**

   * `include` → whitelist fields.
   * `exclude` → blacklist fields.
   * Combine with `exclude_unset` / `exclude_defaults`.

5. **🧭 Aliases**

   * Use `by_alias=True` to output external names.

6. **🧪 Custom Serializers**

   * `@field_serializer` / `@model_serializer` → fine control of output format.

7. **🧱 Nested Models**

   * Recursively serializes models, lists, dicts.
   * Filters can target sub-objects.

8. **🕒 Built-in Types**

   * Dates → ISO strings in `json` mode.
   * UUID, Decimal, bytes → strings in `json` mode.

9. **🛡️ Privacy**

   * Exclude or mask secrets.

10. **⚙️ Performance**

* Use `exclude_unset` for PATCH.
* Combine `exclude_defaults` + `exclude_none` for compact payloads.

---



In [2]:
from pydantic import BaseModel, Field, field_serializer
from datetime import datetime
from uuid import uuid4
from decimal import Decimal

class User(BaseModel):
    id: int
    username: str = Field(alias="user_name")
    created_at: datetime
    balance: Decimal
    password: str

    # Mask password when dumping
    @field_serializer("password")
    def hide_password(self, v: str):
        return "****"

# ✅ Create an object
user = User(
    id=1,
    user_name="Mukesh",
    created_at=datetime.utcnow(),
    balance=Decimal("1234.50"),
    password="supersecret"
)

# 1️⃣ Dump as dict (Python mode)
print("Dict dump:", user.model_dump(mode="python"))

# 2️⃣ Dump as JSON (JSON mode)
print("JSON dump:", user.model_dump_json())

# 3️⃣ Exclude field
print("Exclude password:", user.model_dump(exclude={"password"}))

# 4️⃣ Use alias
print("With alias:", user.model_dump(by_alias=True))


Dict dump: {'id': 1, 'username': 'Mukesh', 'created_at': datetime.datetime(2025, 9, 9, 5, 7, 41, 509562), 'balance': Decimal('1234.50'), 'password': '****'}
JSON dump: {"id":1,"username":"Mukesh","created_at":"2025-09-09T05:07:41.509562","balance":"1234.50","password":"****"}
Exclude password: {'id': 1, 'username': 'Mukesh', 'created_at': datetime.datetime(2025, 9, 9, 5, 7, 41, 509562), 'balance': Decimal('1234.50')}
With alias: {'id': 1, 'user_name': 'Mukesh', 'created_at': datetime.datetime(2025, 9, 9, 5, 7, 41, 509562), 'balance': Decimal('1234.50'), 'password': '****'}
